In [55]:
from dotenv import load_dotenv
import dspy
from pydantic import BaseModel, Field

load_dotenv("../.env")

True

In [10]:
turbo = dspy.OpenAI(model="gpt-3.5-turbo-instruct")

In [11]:
class QuestionAnswer(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField()


class ZeroShot(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predictor = dspy.Predict(QuestionAnswer)

    def forward(self, question):
        return self.predictor(question=question)

In [30]:
with dspy.context(lm=turbo):
    qa = dspy.Predict("statement -> sentiment")
    response = qa(statement="Felling good today")
    print(response.sentiment)

Positive


In [28]:
with dspy.context(lm=turbo):
    document = """
Large Language Models (LLMs) are advanced AI models that have revolutionized \
natural language processing (NLP) tasks by significantly expanding the data \
used for training and inference. These models, such as GPT-4, are trained on \
vast amounts of data, enabling them to generate text, handle text and image \
inputs (multimodal approach), and perform tasks like translation, text \
summarization, sentiment analysis, and code generation. LLMs are \
characterized by their massive size, with billions and even trillions of \
parameters, and are trained using self-supervised or semi-supervised learning \
methodologies. They have broad applicability across various industries, from \
marketing and sales to healthcare and finance, transforming processes, \
improving customer experiences, and enabling more efficient decision-making. \
Additionally, LLMs are poised to reshape the way we interact with technology \
and access information, making them a pivotal part of the modern digital \
landscape.
    """
    summarize = dspy.ChainOfThought("document -> summary")
    response = summarize(document=document)
    print(response.summary)
    print("Rationale:", response.rationale)

Large Language Models (LLMs) are advanced AI models that have revolutionized natural language processing (NLP) tasks. They are trained on vast amounts of data and have billions and even trillions of parameters. LLMs have broad applicability across various industries and are poised to reshape the way we interact with technology and access information.
Rationale: understand the significance of Large Language Models (LLMs). We will explore their capabilities, training methodologies, and potential impact on various industries.


In [31]:
with dspy.context(lm=turbo):
    qa = dspy.ChainOfThought("question -> answer", n=5)
    response = qa(question="What is the capital of Brazil?")
    print(response.completions.answer)
    print("Rationale:", response.rationale)

['Brasília', 'The capital of Brazil is Brasília.', 'Brasília', 'Brasília', 'Brasilia']
Rationale: find the capital of Brazil. We know that Brazil is a big country in South America. We also know that each country has a capital city. So, the capital of Brazil must be a city in South America. We know that the country's capital is Brasília, so that must be the capital of Brazil.


In [14]:
with dspy.context(lm=turbo):
    model = ZeroShot()
    question = "What is the capital of Brazil?"
    response = model(question=question)
    print(response.answer)
    # Inspect
    turbo.inspect_history(n=1)

Brasília




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Answer: ${answer}

---

Question: What is the capital of Brazil?
Answer: Brasília





In [33]:
class Emotion(dspy.Signature):
    """Classify emotion among sadness, joy, love, anger, fear, surprise."""

    sentence = dspy.InputField()
    sentiment = dspy.OutputField()


sentence = "Felling good today"

with dspy.context(lm=turbo):
    classify = dspy.Predict(Emotion)
    response = classify(sentence=sentence)
    print(response.sentiment)

joy


In [36]:
class BasicQA(dspy.Signature):
    """Answer the quesions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")


with dspy.context(lm=turbo):
    qa = dspy.ChainOfThought(BasicQA)
    hint = "It's what you often see during a sunny day."
    response = qa(question="What is the color of the sky?", hint=hint)
    print(response.answer)
    # Inspect
    turbo.inspect_history(n=1)
    print("Rationale:", response.rationale)

blue




Answer the quesions with short factoid answers.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: often between 1 and 5 words

---

Question: What is the color of the sky?
Reasoning: Let's think step by step in order to determine the color of the sky. We know that the sky is blue because of the way the atmosphere scatters sunlight.
Answer: blue



Rationale: determine the color of the sky. We know that the sky is blue because of the way the atmosphere scatters sunlight.


In [41]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")


with dspy.context(lm=turbo):
    pot = dspy.ProgramOfThought(GenerateAnswer)
    question = "Sarah has 5 apples. She buys 7 more apples from the store. How many apples does Sarah have now?"
    result = pot(question=question)
    print(result.answer)

12


In [47]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")


with dspy.context(lm=turbo):
    react_module = dspy.ReAct(BasicQA)
    question = "What is the color of the sky?"
    result = react_module(question=question)
    print(result.answer)
    print("Rationale:", response.rationale)

blue
Rationale: determine the color of the sky. We know that the sky is blue because of the way the atmosphere scatters sunlight.


In [70]:
class Input(BaseModel):
    context: str = Field(description="The context for the question")
    query: str = Field(description="The question to be answered")


class Output(BaseModel):
    answer: str = Field(description="The answer for the question")
    confidence: float = Field(
        ge=0, le=1, description="The confidence score for the answer"
    )


class QASignature(dspy.Signature):
    """Answer the question based on the context and query provided, and on the scale of 10 tell how confident you are about the answer."""

    input: Input = dspy.InputField()
    output: Output = dspy.OutputField()


with dspy.context(lm=turbo):
    predictor = dspy.TypedPredictor(QASignature)
    doc_query_pair = Input(
        context="The quick brown fox jumps over the lazy dog",
        query="What does the fox jumps over?",
    )
    prediction = predictor(input=doc_query_pair)
    answer = prediction.output.answer
    confidence_score = prediction.output.confidence
    print(answer)
    print(confidence_score)

The fox jumps over the lazy dog
0.9
